In [1]:
import optuna
import math
import numpy as np
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import pickle
from collections import defaultdict


In [2]:
oer_desc_data = pickle.load(open("racs_and_desc/data_mc3_lc0.p", "rb"))

oer_desc_vals = []
racs = []
catalyst_names = []

for name in oer_desc_data:
    oer_desc_vals.append(oer_desc_data[name][0])
    racs.append(oer_desc_data[name][1])
    catalyst_names.append(name)
racs = np.asarray(racs)

In [ ]:
from sklearn.model_selection import KFold
#Grabbing a sklearn Classification dataset:
#Step 1. Define an objective function to be maximized.
def objective(trial):

    regressor_name = trial.suggest_categorical("regressor", ["SVR", "RandomForest", "KRR_RBF", "KRR_Linear"])
    
    # Step 2. Setup values for the hyperparameters:
    if regressor_name == "SVR":
        
        svr_degree = trial.suggest_int("svr_degree", 2, 5)
        epsilon = trial.suggest_float("epsilon", 1e-3, 1, log=True)
        regularization = trial.suggest_float("reg_svr", 1e-3, 10, log=True)
        regressor_obj = SVR(degree=svr_degree, epsilon=epsilon, C=regularization)
    elif regressor_name == "RandomForest":
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 10, 1000)
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        regressor_obj = RandomForestRegressor(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators, random_state=42
        )
    elif regressor_name == "KRR_RBF":
        krr_alpha = trial.suggest_float("krr_alpha", 0.01, 1000)
        regressor_obj = KernelRidge(kernel="rbf", alpha=krr_alpha)
        
    elif regressor_name == "KRR_Linear":
        krr_l_alpha = trial.suggest_float("krr_l_alpha", 0.01, 1000)
        krr_degree = trial.suggest_int("krr_degree", 2, 5)
        regressor_obj = KernelRidge(kernel="linear", alpha=krr_l_alpha)

    # Step 3: Scoring method:

    preds = []
    test_vals = []
    errs = []
    loo = LeaveOneOut()
    for train_index, test_index in loo.split(racs, oer_desc_vals):
        sc = StandardScaler()
        X_train, X_test = racs[train_index], racs[test_index]
        y_train, y_test = np.asarray(oer_desc_vals)[train_index], np.asarray(oer_desc_vals)[test_index]
        scaled_X_train = sc.fit_transform(X_train)
        scaled_X_test = sc.transform(X_test)

        model = regressor_obj

        model.fit(scaled_X_train, y_train)

        pred = model.predict(scaled_X_test,)
        preds.append(pred[0])
        test_vals.append(y_test[0])
    accuracy_rmse = math.sqrt(mean_squared_error(np.asarray(test_vals), np.asarray(preds)))
    return accuracy_rmse

# Step 4: Running it
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)

[I 2021-08-27 21:16:17,541] A new study created in memory with name: no-name-e3fc76ea-61a2-4ab7-8312-4c8539dea892
[I 2021-08-27 21:16:18,487] Trial 0 finished with value: 1.328771828873278 and parameters: {'regressor': 'KRR_RBF', 'krr_alpha': 429.3434885983287}. Best is trial 0 with value: 1.328771828873278.
[I 2021-08-27 21:16:18,892] Trial 1 finished with value: 1.5312765213727502 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 318.72223810405603, 'krr_degree': 4}. Best is trial 0 with value: 1.328771828873278.
[I 2021-08-27 21:17:54,893] Trial 2 finished with value: 0.08116611584067433 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 194, 'rf_max_depth': 7}. Best is trial 2 with value: 0.08116611584067433.
[I 2021-08-27 21:17:55,443] Trial 3 finished with value: 1.5320949203016965 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 651.7120508011338, 'krr_degree': 4}. Best is trial 2 with value: 0.08116611584067433.
[I 2021-08-27 21:20:51,190] Trial

[I 2021-08-27 22:11:19,562] Trial 35 finished with value: 0.0826022305891878 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 548, 'rf_max_depth': 14}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 22:12:37,565] Trial 36 finished with value: 0.08067614165210904 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 269, 'rf_max_depth': 5}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 22:13:33,074] Trial 37 finished with value: 0.08061887622785947 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 174, 'rf_max_depth': 6}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 22:13:33,467] Trial 38 finished with value: 1.5277184733407976 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 2.2544486233280168, 'krr_degree': 3}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 22:15:15,924] Trial 39 finished with value: 0.08443611411075866 and parameters: {'regressor': 'RandomForest'

[I 2021-08-27 22:53:54,852] Trial 71 finished with value: 0.08041736300804221 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 404, 'rf_max_depth': 6}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 22:55:39,783] Trial 72 finished with value: 0.08430718927139522 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 459, 'rf_max_depth': 4}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 22:57:39,736] Trial 73 finished with value: 0.08039540577062496 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 431, 'rf_max_depth': 6}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 23:02:13,129] Trial 74 finished with value: 0.08130417411923153 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 911, 'rf_max_depth': 7}. Best is trial 21 with value: 0.08031984722570795.
[I 2021-08-27 23:03:41,388] Trial 75 finished with value: 0.08062170350386749 and parameters: {'regressor': 'RandomForest', 'rf_

[I 2021-08-27 23:50:17,396] Trial 107 finished with value: 0.08119262771169805 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 213, 'rf_max_depth': 7}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-27 23:50:17,696] Trial 108 finished with value: 1.531211184352513 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 287.2606086787559, 'krr_degree': 4}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-27 23:51:21,835] Trial 109 finished with value: 0.08063299668673074 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 252, 'rf_max_depth': 5}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-27 23:51:23,202] Trial 110 finished with value: 0.0859558237412882 and parameters: {'regressor': 'SVR', 'svr_degree': 2, 'epsilon': 0.008142380458782917, 'reg_svr': 1.3043027043920714}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-27 23:52:56,417] Trial 111 finished with value: 0.08034056859745903 and parameter

[I 2021-08-28 00:35:03,062] Trial 143 finished with value: 0.08249446726234386 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 325, 'rf_max_depth': 27}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-28 00:36:45,698] Trial 144 finished with value: 0.08037282014181667 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 368, 'rf_max_depth': 6}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-28 00:38:09,208] Trial 145 finished with value: 0.08108854493328128 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 279, 'rf_max_depth': 7}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-28 00:39:33,840] Trial 146 finished with value: 0.0805520248215726 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 332, 'rf_max_depth': 5}. Best is trial 82 with value: 0.08031282560857571.
[I 2021-08-28 00:40:46,363] Trial 147 finished with value: 0.08039342880245684 and parameters: {'regressor': 'RandomForest',

...

[I 2021-08-28 08:50:30,928] Trial 501 finished with value: 0.08066355951629177 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 361, 'rf_max_depth': 5}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 08:52:05,971] Trial 502 finished with value: 0.08105384299544761 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 316, 'rf_max_depth': 7}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 08:52:06,272] Trial 503 finished with value: 1.5329938653765838 and parameters: {'regressor': 'KRR_Linear', 'krr_l_alpha': 985.8709837857245, 'krr_degree': 5}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 08:53:25,911] Trial 504 finished with value: 0.0803956454018164 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 283, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 08:55:02,421] Trial 505 finished with value: 0.08038199075082587 and parameters: {'regressor': 'Random

[I 2021-08-28 09:39:09,565] Trial 536 finished with value: 0.08034853888055091 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 378, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 09:40:45,778] Trial 537 finished with value: 0.08107009803697886 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 320, 'rf_max_depth': 7}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 09:42:23,664] Trial 538 finished with value: 0.08036410294563517 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 351, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 09:43:48,330] Trial 539 finished with value: 0.0805439250113772 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 333, 'rf_max_depth': 5}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 09:45:06,190] Trial 540 finished with value: 0.08037661444009002 and parameters: {'regressor': 'RandomFores

[I 2021-08-28 10:28:10,749] Trial 572 finished with value: 0.0811339711706766 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 350, 'rf_max_depth': 7}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 10:29:35,308] Trial 573 finished with value: 0.08033852856348157 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 304, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 10:30:57,093] Trial 574 finished with value: 0.08033424987362525 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 293, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 10:32:28,369] Trial 575 finished with value: 0.08031608924495495 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 328, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 10:34:13,179] Trial 576 finished with value: 0.0811339711706766 and parameters: {'regressor': 'RandomForest

[I 2021-08-28 11:14:51,945] Trial 608 finished with value: 0.08030867559930044 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 320, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 11:16:30,825] Trial 609 finished with value: 0.08103408129824242 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 291, 'rf_max_depth': 7}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 11:18:18,258] Trial 610 finished with value: 0.08036291636146678 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 342, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 11:19:46,919] Trial 611 finished with value: 0.08034404927053032 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 272, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 11:21:51,595] Trial 612 finished with value: 0.08113315705712687 and parameters: {'regressor': 'RandomFore

[I 2021-08-28 12:09:44,809] Trial 644 finished with value: 0.08034901756527182 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 278, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 12:11:19,236] Trial 645 finished with value: 0.08032623004996499 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 334, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 12:13:02,585] Trial 646 finished with value: 0.08037736323413586 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 366, 'rf_max_depth': 6}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 12:14:37,408] Trial 647 finished with value: 0.08104864128382408 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 313, 'rf_max_depth': 7}. Best is trial 241 with value: 0.08028504564222302.
[I 2021-08-28 12:16:00,071] Trial 648 finished with value: 0.08033835138531178 and parameters: {'regressor': 'RandomFore